In [16]:
from typing import TypedDict, List, Optional, Union
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field,EmailStr,field_validator
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage,BaseMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic_extra_types.phone_numbers import PhoneNumber
from langgraph.graph import StateGraph,add_messages,START,END
from langchain_core.agents import AgentAction,AgentFinish
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
from pydantic import BaseModel, Field, EmailStr
from typing import List, Optional, Dict, Any
from langchain_core.messages import BaseMessage
import sqlite3

class PatientInfo(BaseModel):
    name: Optional[str] = None
    age: Optional[int] = None
    gender: Optional[str] = None
    email: Optional[EmailStr] = None
    phone_number: Optional[str] = None
    address: Optional[str] = None

class AgentCall(BaseModel):
    tool_name: str
    input: Dict[str, Any]
    output: Optional[Any] = None
    status: str = "pending"

class AgentState(BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    agent_calls: List[AgentCall] = Field(default_factory=list)
    patient_info: PatientInfo = Field(default_factory=PatientInfo)
    current_agent: Optional[str] = "scheduler"


In [ ]:
from langchain.tools import tool
from datetime import datetime
from typing import List



@tool
def book_slot(slot_id: int):
    """Books available slot on today using slot_id"""
    # These should be already defined in your file:
    slot_conn = sqlite3.connect('slot_status.db')
    slot_cursor = slot_conn.cursor()
    today = datetime.today().date().isoformat()
    slot_cursor.execute('SELECT status FROM slots WHERE id = ? AND date = ?', (slot_id, today))
    row = slot_cursor.fetchone()
    if row is None:
        return f"❌ Slot ID {slot_id} is invalid for today."
    elif row[0] == 1:
        return f"⚠️ Slot ID {slot_id} is already booked."
    else:
        slot_cursor.execute('UPDATE slots SET status = 1 WHERE id = ? AND date = ?', (slot_id, today))
        slot_conn.commit()
        return f"✅ Slot ID {slot_id} booked successfully."

@tool
def view_available_slots() -> List:
    """Lists today's available slots"""
    # These should be already defined in your file:
    slot_conn = sqlite3.connect('slot_status.db')
    slot_cursor = slot_conn.cursor()
    today = datetime.today().date().isoformat()
    slot_cursor.execute('SELECT id, slot FROM slots WHERE status = 0 AND date = ?', (today,))
    slots = slot_cursor.fetchall()
    if not slots:
        return ["❌ No available slots for today."]
    return [{"id": slot[0], "time": slot[1]} for slot in slots]


In [19]:
view_available_slots("entry")


[{'id': 25, 'time': '09:00 AM - 09:20 AM'},
 {'id': 26, 'time': '09:20 AM - 09:40 AM'},
 {'id': 27, 'time': '09:40 AM - 10:00 AM'},
 {'id': 28, 'time': '10:00 AM - 10:20 AM'},
 {'id': 29, 'time': '10:20 AM - 10:40 AM'},
 {'id': 30, 'time': '10:40 AM - 11:00 AM'},
 {'id': 31, 'time': '11:00 AM - 11:20 AM'},
 {'id': 32, 'time': '11:20 AM - 11:40 AM'},
 {'id': 33, 'time': '11:40 AM - 12:00 PM'},
 {'id': 34, 'time': '12:00 PM - 12:20 PM'},
 {'id': 35, 'time': '12:20 PM - 12:40 PM'},
 {'id': 36, 'time': '12:40 PM - 01:00 PM'},
 {'id': 37, 'time': '01:00 PM - 01:20 PM'},
 {'id': 38, 'time': '01:20 PM - 01:40 PM'},
 {'id': 39, 'time': '01:40 PM - 02:00 PM'},
 {'id': 40, 'time': '02:00 PM - 02:20 PM'},
 {'id': 42, 'time': '02:40 PM - 03:00 PM'},
 {'id': 44, 'time': '03:20 PM - 03:40 PM'},
 {'id': 45, 'time': '03:40 PM - 04:00 PM'},
 {'id': 46, 'time': '04:00 PM - 04:20 PM'},
 {'id': 47, 'time': '04:20 PM - 04:40 PM'},
 {'id': 48, 'time': '04:40 PM - 05:00 PM'}]

In [20]:
tools = [view_available_slots,book_slot]

In [21]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-001", temperature=0)

In [22]:
from langgraph.prebuilt import create_react_agent

In [23]:
system_prompt = """You are an intelligent appointment scheduling assistant.

You have access to two tools:
1. `view_available_slots()` → returns a list of today's available time slots.
2. `book_slot(slot_id: int)` → books a slot for today using its ID. It may return:
   - '✅ Slot ID slot_id booked successfully.'
   - '❌ Slot ID slot_id is invalid for today.'
   - '⚠️ Slot ID slot_id is already booked.'

Your workflow:
- Always start by calling `view_available_slots()` to check today’s free slots.
- If no slots are available, inform the user politely.
- If exactly one slot is available, automatically call `book_slot(slot_id)` to book it.
- If multiple slots are available:
   - List them clearly with their `id` and `time`.
   - Ask the user to choose a slot ID.
- When the user provides a slot ID, call `book_slot(slot_id)` using the integer.

Rules:
- Only use slot IDs from `view_available_slots()` — never invent or guess.
- `slot_id` must be an integer when calling `book_slot(slot_id: int)`.
- If the slot is already booked or invalid:
   - Notify the user.
   - Ask for a valid slot ID from the list.

Interaction guidelines:
- Clearly present all available slots with IDs and times.
- Confirm when a booking is successful.
- Handle errors gracefully and helpfully.

Always ensure a smooth and helpful experience for the user.
"""

In [24]:
agent  = create_react_agent(model = llm , tools= tools , name = "Schedulare agent" , prompt=system_prompt , debug=True)

In [30]:
user_input = input("How can i help you: ")
for step in  agent.stream({
    "messages" : [{
        "role" : "user",
        "content" : "i would like to book the id 25 slot"
    }]
}):
           print(step)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [{'content': 'i would like to book the id 25 slot',
               'role': 'user'}]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [{'content': 'i would like to book the id 25 slot', 'role': 'user'}]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='i would like to book the id 25 slot', additional_kwargs={}, response_metadata={}, id='8edbc484-cedc-47b6-930e-39466915ce47')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='i would like to book the id 25 slot', additional_kwargs={}, response_metadata={}, id='8edbc484-cedc-47b6-930e-39466915ce47')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'function_call': {'name': 'book_slot', 'arguments': '{"slot_id": 25.0}'}}, r

In [33]:

while True:
    if user_input != "exit":
           user_input = input("what ")
    else:
          break



In [ ]:
result =  agent.invoke(input={
    "messages" : [{
        "role" : "user",
        "content" : "i would like to book the id 42 slot"
        }]
    })


    

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [{'content': 'i would like to book the id 42 slot',
               'role': 'user'}]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [{'content': 'i would like to book the id 42 slot', 'role': 'user'}]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='i would like to book the id 42 slot', additional_kwargs={}, response_metadata={}, id='a375d34d-f188-4849-996a-0cf992ae966f')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='i would like to book the id 42 slot', additional_kwargs={}, response_metadata={}, id='a375d34d-f188-4849-996a-0cf992ae966f')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'function_call': {'name': 'book_slot', 'arguments': '{"slot_id": 42.0}'}}, r

In [27]:
import pprint

pprint.pprint(result)

{'messages': [HumanMessage(content='hi i want to book an appointment for me , can you tell me which slots are available', additional_kwargs={}, response_metadata={}, id='7bf59ae2-9662-4ff4-b035-3a75cc8ef716'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'view_available_slots', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, name='Schedulare agent', id='run--11462b1c-4db5-4fac-89e2-c923f57cea2f-0', tool_calls=[{'name': 'view_available_slots', 'args': {}, 'id': '3d1e7085-df77-4cbd-875e-b26accfb58ef', 'type': 'tool_call'}], usage_metadata={'input_tokens': 404, 'output_tokens': 5, 'total_tokens': 409, 'input_token_details': {'cache_read': 0}}),
              ToolMessage(content='[{"id": 25, "time": "09:00 AM - 09:20 AM"}, {"id": 26, "time": "09:20 AM - 09:40 AM"}, {"id": 27, "time": "09:40 AM - 10:00 AM"}, {"id": 28

In [ ]:
llm_with_tools= llm.bind_tools(tools = tools)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


scheduler_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(
        content=(
            "You are an intelligent appointment scheduling assistant.\n\n"
            "You have access to two tools:\n"
            "{tools}"
            "1. `view_avadilable_slots()` → returns a list of available for today"
            
            "2. `book_slot(slot_id: int)` → books a slot for today by its ID. It returns messages like:\n"
            "   - '✅ Slot ID {slot_id} booked successfully.'\n"
            "   - '❌ Slot ID {slot_id} is invalid for today.'\n"
            "   - '⚠️ Slot ID {slot_id} is already booked.'\n\n"

            "Your workflow:\n"
            "- Always start by calling `view_available_slots()` to check today’s free slots.\n"
            "- If no slots are available, inform the user politely.\n"
            "- If exactly one slot is available, automatically call `book_slot(slot_id)` to book it.\n"
            "- If multiple slots are available, list them with their `id` and `time`, then ask the user to choose a slot ID.\n"
            "- When the user replies with a slot ID, call `book_slot(slot_id)` with the correct integer.\n\n"

            "Rules:\n"
            "- The `slot_id` must be passed as an integer to `book_slot(slot_id: int)`.\n"
            "- Do not guess or invent slot IDs — always rely on what `view_available_slots()` returned.\n"
            "- If a slot is already booked or invalid, notify the user and ask for another valid ID from the list."
        )
    ),
    AIMessage(
        content="I'm here to help you book your appointment. Let me first check the available slots for today..."
    ),
    MessagesPlaceholder(variable_name="user_input"),
    SystemMessage(
        content=(
            "Make sure to:\n"
            "- Clearly list available time slots with their IDs when offering choices.\n"
            "- Confirm with the user once a booking is successful.\n"
            "- Handle invalid or already-booked slot IDs gracefully.\n\n"
            "Use the tools responsibly and ensure the interaction is smooth and helpful for the user."
        )
    )
])


In [ ]:
res = llm_with_tools.invoke("which tools do you have?")
pprint(res)

AIMessage(content='I have access to the following tools:\n\n`default_api`:\n```python\ndef view_available_slots(\n) -> dict:\n  """Lists today\'s available slots\n\n  Args:\n  """\n\n\ndef book_slot(\n    slot_id: int,\n) -> dict:\n  """Books available slot on today using slot_id\n\n  Args:\n    slot_id: \n  """\n```', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run--168a48de-3575-41cf-883d-7cc97afc124f-0', usage_metadata={'input_tokens': 37, 'output_tokens': 93, 'total_tokens': 130, 'input_token_details': {'cache_read': 0}})


In [ ]:
chain = scheduler_prompt | llm_with_tools

In [ ]:
from pprint import pprint

In [ ]:
user_input = HumanMessage("hi i want to book an appointment for me , can you tell me which slots are available")

In [ ]:
res = chain.invoke([user_input])
pprint(res)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'view_available_slots', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run--7b8af6cd-a0b3-49bc-8e32-71d5bba2b921-0', tool_calls=[{'name': 'view_available_slots', 'args': {}, 'id': '1ee06e7a-ec11-43cb-a011-63adfc27c309', 'type': 'tool_call'}], usage_metadata={'input_tokens': 435, 'output_tokens': 5, 'total_tokens': 440, 'input_token_details': {'cache_read': 0}})


In [ ]:
print(book_slot)
print(view_available_slots)

name='book_slot' description='Books available slot on today using slot_id' args_schema=<class 'langchain_core.utils.pydantic.book_slot'> func=<function book_slot at 0x7febf5bc9cf0>
name='view_available_slots' description="Lists today's available slots" args_schema=<class 'langchain_core.utils.pydantic.view_available_slots'> func=<function view_available_slots at 0x7febf5877e20>


In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import AIMessage, HumanMessage

def scheduler_logic(state: AgentState) -> AgentState:
    last_user_msg = next((m for m in reversed(state.messages) if isinstance(m, HumanMessage)), None)
    if last_user_msg:
        content = last_user_msg.content.lower()

        llm_prompt = 

        if "available" in content:
            output = view_available_slots.invoke({})  # ✅ fixed here
            state.messages.append(AIMessage(content=str(output)))
            state.agent_calls.append(AgentCall(
                tool_name="view_available_slots",
                input={},
                output=output,
                status="completed"
            ))

        elif "book" in content:
            slot_id = 1  # Placeholder
            output = book_slot.invoke({"slot_id": slot_id})  # ✅ fixed here
            state.messages.append(AIMessage(content=str(output)))
            state.agent_calls.append(AgentCall(
                tool_name="book_slot",
                input={"slot_id": slot_id},
                output=output,
                status="completed"
            ))

        else:
            state.messages.append(AIMessage(content="I can help you view or book slots. Try asking: 'What slots are available?'"))

    return state




In [ ]:
from langgraph.graph import StateGraph, END

builder = StateGraph(AgentState)
builder.add_node("scheduler", scheduler_logic)
builder.set_entry_point("scheduler")
builder.set_finish_point("scheduler")
graph = builder.compile()


In [ ]:
from langchain_core.messages import HumanMessage

initial_state = AgentState(
    messages=[HumanMessage(content="What slots are available today?, if any available book an appointment in the first available slot.")]
)

final_state = graph.invoke(initial_state)

for msg in final_state:
    print(final_state[msg])


[HumanMessage(content='What slots are available today?, if any available book an appointment in the first available slot.', additional_kwargs={}, response_metadata={}), AIMessage(content="[{'id': 25, 'time': '09:00 AM - 09:20 AM'}, {'id': 26, 'time': '09:20 AM - 09:40 AM'}, {'id': 27, 'time': '09:40 AM - 10:00 AM'}, {'id': 28, 'time': '10:00 AM - 10:20 AM'}, {'id': 29, 'time': '10:20 AM - 10:40 AM'}, {'id': 30, 'time': '10:40 AM - 11:00 AM'}, {'id': 31, 'time': '11:00 AM - 11:20 AM'}, {'id': 32, 'time': '11:20 AM - 11:40 AM'}, {'id': 33, 'time': '11:40 AM - 12:00 PM'}, {'id': 34, 'time': '12:00 PM - 12:20 PM'}, {'id': 35, 'time': '12:20 PM - 12:40 PM'}, {'id': 36, 'time': '12:40 PM - 01:00 PM'}, {'id': 37, 'time': '01:00 PM - 01:20 PM'}, {'id': 38, 'time': '01:20 PM - 01:40 PM'}, {'id': 39, 'time': '01:40 PM - 02:00 PM'}, {'id': 40, 'time': '02:00 PM - 02:20 PM'}, {'id': 41, 'time': '02:20 PM - 02:40 PM'}, {'id': 42, 'time': '02:40 PM - 03:00 PM'}, {'id': 43, 'time': '03:00 PM - 03:20 